In [1]:
from helpers import *
from model import Model
import pandas as pd

import pickle

/Users/raficg/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
data = pd.read_csv('../data/train/cancellations.csv', parse_dates=['full_date'])
data.replace(['City Hotel', 'Resort Hotel'],  [1, 0], inplace=True)

In [3]:
X, y, cols = clean_data(data)

ValueError: not enough values to unpack (expected 3, got 2)

In [5]:
cols

Index(['hotel', 'room_difference', 'booking_changes', 'total_nights_2',
       'total_nights_3', 'total_nights_4', 'total_nights_5', 'total_nights_6',
       'total_nights_7', 'total_nights_8', 'total_of_special_requests_1',
       'total_of_special_requests_2', 'market_segment_Direct',
       'market_segment_Groups', 'market_segment_Offline TA/TO',
       'market_segment_Online TA', 'party_size_2.0', 'party_size_3.0'],
      dtype='object')

In [4]:
model = Model()

In [5]:
(resort_bookings_by_day, 
resort_cancellations_by_day, 
city_bookings_by_day, 
city_cancellations_by_day) = get_day_counts(data)

In [6]:
data

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,total_of_special_requests,reservation_status,reservation_status_date,total_nights,party_size,is_family,room_difference,had_agent,had_company,full_date
0,1,0,164,2015,October,40,2,0,2,1,...,0,Check-Out,2015-10-04,2,1.0,0,1,0,0,2015-10-02
1,0,0,24,2016,November,47,18,2,5,2,...,3,Check-Out,2016-11-25,7,2.0,0,1,0,0,2016-11-18
2,1,1,222,2017,August,35,31,0,3,2,...,3,Canceled,2017-05-05,3,2.0,0,0,0,0,2017-08-31
3,0,1,267,2015,September,39,26,2,5,2,...,0,Canceled,2015-06-17,7,2.0,0,0,0,0,2015-09-26
4,0,0,327,2016,September,38,14,0,4,2,...,0,Check-Out,2016-09-18,4,2.0,0,1,0,0,2016-09-14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94465,1,0,0,2015,September,38,16,0,1,2,...,0,Check-Out,2015-09-17,1,2.0,0,1,0,0,2015-09-16
94466,1,0,4,2017,August,33,15,0,1,2,...,0,Check-Out,2017-08-16,1,4.0,1,0,0,0,2017-08-15
94467,1,0,165,2017,July,27,7,1,2,2,...,1,Check-Out,2017-07-10,3,2.0,0,0,0,0,2017-07-07
94468,0,0,162,2015,July,28,7,2,7,2,...,1,Check-Out,2015-07-16,9,2.0,0,0,0,0,2015-07-07


In [7]:
resorts = pd.Series(data=(np.array(resort_bookings_by_day) + 
                          np.array(resort_cancellations_by_day)), 
                    index=sorted(data['full_date'].unique()))
citys = pd.Series(data=(np.array(city_bookings_by_day) + 
                          np.array(city_cancellations_by_day)), 
                    index=sorted(data['full_date'].unique()))

In [8]:
model.fit_classifier(X, y)
model.fit_demand_forecaster(resorts, citys)

In [9]:
model.city_dates

,ds,y,month,day_of_month,year
15,2015-07-16,95.0,7,16,2015
16,2015-07-17,168.0,7,17,2015
17,2015-07-18,230.0,7,18,2015
18,2015-07-19,191.0,7,19,2015
19,2015-07-20,174.0,7,20,2015
...,...,...,...,...,...
788,2017-08-27,345.0,8,27,2017
789,2017-08-28,338.0,8,28,2017
790,2017-08-29,296.0,8,29,2017
791,2017-08-30,276.0,8,30,2017


In [10]:
citys

2015-07-01     64.0
2015-07-02    108.0
2015-07-03    121.0
2015-07-04     91.0
2015-07-05     61.0
              ...  
2017-08-27    345.0
2017-08-28    338.0
2017-08-29    296.0
2017-08-30    276.0
2017-08-31    285.0
Length: 793, dtype: float64

In [4]:
dates = pd.DatetimeIndex(data['full_date'])
#probs, dems = model.predict_proba(X, dates)

In [12]:
with open('model.pkl', 'wb') as f:
    # Write the model to a file.
    pickle.dump(model, f)    

In [15]:
# with open('model.pkl', 'rb') as f:
#     # Write the model to a file.
#     model = pickle.load(f)    

In [9]:
dates.strftime('%Y-%m-%d')

Index(['2015-10-02', '2016-11-18', '2017-08-31', '2015-09-26', '2016-09-14',
       '2017-07-11', '2017-02-01', '2015-10-17', '2015-11-14', '2016-09-13',
       ...
       '2017-07-04', '2016-10-23', '2016-08-26', '2016-01-26', '2016-12-27',
       '2015-09-16', '2017-08-15', '2017-07-07', '2015-07-07', '2017-03-17'],
      dtype='object', name='full_date', length=94470)